# Anomaly detection 2

In this notebook we are going to perform anomaly detection on the dataset data_clean.xlsx using : iForest, OneClassSVM and DBSCAN. We are going to detect anomalies in the entire dataset using all the variables after encoding the categorical ones.

In [1]:
## Importing necessary libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from numpy import where

In [10]:
#Load data

file_name = ".\\data_clean.xlsx"
xl_file = pd.ExcelFile(file_name,engine = 'openpyxl')
df = xl_file.parse('Sheet1')

## Data preprocessing

In [11]:
# Null values 
df.dropna(subset=['data'], inplace=True)

##Handling units
# We are going to convert everything that is MWh to KWh and  Nm³ , CCF , l, kl to m³

df['unit']=df['unit'].astype('string')

#df2 = df
#print(len(df2[df2['unit']=='MWh']))
for i, row in df.iterrows():
    # 1MWh = 1000KWh
    if row['unit'] == 'MWh':
        df.at[i, 'data'] = row['data']*1000
        df.at[i, 'unit'] = 'kWh'

    #we will consider that we work in stadard condition (T=273.15K et P=101325 Pa) and 1Nm³ = 1m³
    if row['unit'] == 'Nm³':
        df.at[i, 'unit'] = 'm³'

    # 1CCF = 2,83m³
    if row['unit'] == 'CCF':
        df.at[i, 'data'] = row['data']*2.83
        df.at[i, 'unit'] = 'm³'    

    # 1l = 0.001m³
    if row['unit'] == 'l':
        df.at[i, 'data'] = row['data']*0.001
        df.at[i, 'unit'] = 'm³'

    #1kl = 1m³
    if row['unit'] == 'kl':
        df.at[i, 'unit'] = 'm³'

    #1ft³ = 0,0283m³
    if row['unit'] == 'ft³':
        df.at[i, 'data'] = row['data']*0.0283
        df.at[i, 'unit'] = 'm³'

    #1therm = 2,851m³
    if row['unit'] == 'therm':
        df.at[i, 'data'] = row['data']*2.851
        df.at[i, 'unit'] = 'm³'


In [4]:
# df.info()

In [13]:
## Handling the date column
# Here we are going to extract features from the 'date' column, which are : year, month, day, day of the year, week of the year,
# day of the week

import datetime
df1 = df
df1['date'] = pd.to_datetime(df1['date'])
df1['date'] = df1['date'].dt.strftime('%d.%m.%Y')
df1['year'] = pd.DatetimeIndex(df1['date']).year
df1['month'] = pd.DatetimeIndex(df1['date']).month
df1['day'] = pd.DatetimeIndex(df1['date']).day
df1['dayofyear'] = pd.DatetimeIndex(df1['date']).dayofyear
df1['weekofyear'] = pd.DatetimeIndex(df1['date']).weekofyear
df1['weekday'] = pd.DatetimeIndex(df1['date']).weekday
df1 = df1.drop(['date'], axis = 1) 

## Encoding categorical variables

df1 = pd.get_dummies(df1, columns=['year'], drop_first=True, prefix='year')

df1 = pd.get_dummies(df1, columns=['month'], drop_first=True, prefix='month')

df1 = pd.get_dummies(df1, columns=['weekday'], drop_first=True, prefix='wday')

df1 = pd.get_dummies(df1, columns=['meterId'], drop_first=True, prefix='mId')

df1 = pd.get_dummies(df1, columns=['Country'], drop_first=True, prefix='Country')

df1 = pd.get_dummies(df1, columns=['mediaType'], drop_first=True, prefix='mediaType')

df1 = pd.get_dummies(df1, columns=['unit'], drop_first=True, prefix='unit')

df1 = pd.get_dummies(df1, columns=['Location_'], drop_first=True, prefix='Location')

df1 = pd.get_dummies(df1, columns=['meterType'], drop_first=True, prefix='meterType')


## Anomaly detection using iForest

In order the specify the “contamination” parameter c we are going to exploit the results obtained in the other study (Anomaly detection 1), we are going to set c as the weighted average of all the fractions of outliers in each mediaType, for the rest of the parameters we are going to use default values

In [15]:
## Estimating the contamination coefficient using the results of the previous method

anomaly_percentage = [0.0007,0.0266,0.0001,0.0382,0.0001,0.0001,0.0004,0.0001,0.0031,0.0023,0.0016,0.0003,0.0011,0.0109,0,0,0.0001,0.0339,0.0342,0.0001]

mediaTypes = list(df.mediaType.unique())

c = 0

for i in range(len(mediaTypes))  :
    c = c + anomaly_percentage[i]*len(df[df['mediaType']==mediaTypes[i]])/len(df)
    
c = np.round(c,4)
c

0.0035

In [16]:
# Preparing the dataset and performing iforest

X = df1.to_numpy()

model=IsolationForest(n_estimators=50, max_samples='auto', contamination=c,max_features=1.0,random_state=42)
pred = model.fit_predict(X)
idxs = []

for i, e in enumerate(pred):
    if e == -1 :
        idxs.append(i)

anomalies = df.filter(items = idxs, axis=0)
anomalies = anomalies.reset_index(drop=True)
anomalies

,date,meterId,Country,mediaType,data,unit,Location_,meterType,year,month,day,dayofyear,weekofyear,weekday
0,26.10.2019,1593483,Austria,Electricity,0.0,kWh,Austria,Sub 2,2019,10,26,299,43,5
1,07.11.2019,1593483,Austria,Electricity,0.0,kWh,Austria,Sub 2,2019,7,11,192,28,3
2,14.11.2019,1593483,Austria,Electricity,0.0,kWh,Austria,Sub 2,2019,11,14,318,46,3
3,21.11.2019,1593483,Austria,Electricity,0.0,kWh,Austria,Sub 2,2019,11,21,325,47,3
4,01.12.2019,1593483,Austria,Electricity,0.0,kWh,Austria,Sub 2,2019,1,12,12,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,27.05.2022,6181293,India,Electricity,45880.0,kWh,India kalwa,Submeter,2022,5,27,147,21,4
1000,29.05.2022,6181293,India,Electricity,19430.0,kWh,India kalwa,Submeter,2022,5,29,149,21,6
1001,22.12.2021,6181294,India,Electricity,0.0,kWh,India kalwa,Submeter,2021,12,22,356,51,2
1002,29.12.2021,6181294,India,Electricity,0.0,kWh,India kalwa,Submeter,2021,12,29,363,52,2


In [26]:
print('the percentage of total : ',round((len(anomalies)/len(df))*100,2),' %')

the percentage of total :  0.35  %


In [28]:
## Choose mediaType

mediaType = 'Electricity'

df_temp = df[df['mediaType']==mediaType]
temp = anomalies[anomalies['mediaType']==mediaType]
print('locations with anomalies and their percentage : \n')
print(temp['Location_'].value_counts().index,'\n')
print(np.round(temp['Location_'].value_counts().to_numpy()*100,2)/len(temp))

locations with anomalies and their percentage : 

Index(['India kalwa', 'China', 'Netherland', 'Austria', 'India GU', 'Portugal',
       'Switzerland Zug', 'United State FH', 'United State JS',
       'United State SQ', 'Switzerland SH', 'United State DA'],
      dtype='object') 

[71.77700348 13.41463415  6.62020906  2.26480836  1.91637631  1.04529617
  0.87108014  0.69686411  0.69686411  0.34843206  0.17421603  0.17421603]


In [29]:
print('the percentage of anomalies in this mediaType : ',round((len(temp)/len(df_temp))*100,2),' %')

the percentage of anomalies in this mediaType :  0.31  %


In [30]:
print('the number of meterIds : ',temp.meterId.nunique())

the number of meterIds :  62


In [31]:
print("****The anomalies in this mediaType****\n")
temp

****The anomalies in this mediaType****



,date,meterId,Country,mediaType,data,unit,Location_,meterType,year,month,day,dayofyear,weekofyear,weekday
0,24.05.2022,1176880,Austria,Electricity,57.00,kWh,Austria,Submeter,2022,5,24,144,21,1
1,25.05.2022,1176880,Austria,Electricity,71.60,kWh,Austria,Submeter,2022,5,25,145,21,2
2,26.05.2022,1176880,Austria,Electricity,27.60,kWh,Austria,Submeter,2022,5,26,146,21,3
3,27.05.2022,1176880,Austria,Electricity,46.00,kWh,Austria,Submeter,2022,5,27,147,21,4
4,28.05.2022,1176880,Austria,Electricity,28.20,kWh,Austria,Submeter,2022,5,28,148,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,24.02.2022,6158185,India,Electricity,-10.21,kWh,India kalwa,Submeter,2022,2,24,55,8,3
1008,26.02.2022,6158185,India,Electricity,-0.05,kWh,India kalwa,Submeter,2022,2,26,57,8,5
1009,02.03.2022,6158185,India,Electricity,-11.41,kWh,India kalwa,Submeter,2022,2,3,34,5,3
1011,15.01.2021,6339688,Switzerland,Electricity,0.21,kWh,Switzerland SH,Sub 2,2021,1,15,15,2,4


## Anomaly detection using OneClassSVM

We are going to specify the parameter nu using the same method as we did for the “contamination” parameter , which gives nu = 0.0035

In [25]:
##Performing OneClassSVM

from sklearn.svm import OneClassSVM

X = df1.to_numpy()

model = OneClassSVM(nu=c)
pred = model.fit_predict(X)
idxs = []

for i, e in enumerate(pred):
    if e == -1 :
        idxs.append(i)

anomalies = df.filter(items = idxs, axis=0)
anomalies = anomalies.reset_index(drop=True)
anomalies

,date,meterId,Country,mediaType,data,unit,Location_,meterType,year,month,day,dayofyear,weekofyear,weekday
0,24.05.2022,1176880,Austria,Electricity,57.00,kWh,Austria,Submeter,2022,5,24,144,21,1
1,25.05.2022,1176880,Austria,Electricity,71.60,kWh,Austria,Submeter,2022,5,25,145,21,2
2,26.05.2022,1176880,Austria,Electricity,27.60,kWh,Austria,Submeter,2022,5,26,146,21,3
3,27.05.2022,1176880,Austria,Electricity,46.00,kWh,Austria,Submeter,2022,5,27,147,21,4
4,28.05.2022,1176880,Austria,Electricity,28.20,kWh,Austria,Submeter,2022,5,28,148,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,26.02.2022,6158185,India,Electricity,-0.05,kWh,India kalwa,Submeter,2022,2,26,57,8,5
1009,02.03.2022,6158185,India,Electricity,-11.41,kWh,India kalwa,Submeter,2022,2,3,34,5,3
1010,24.11.2021,6183595,Singapore,ColdWater,18.30,m³,Singapour,Main meter,2021,11,24,328,47,2
1011,15.01.2021,6339688,Switzerland,Electricity,0.21,kWh,Switzerland SH,Sub 2,2021,1,15,15,2,4


In [32]:
print('the percentage of total : ',round((len(anomalies)/len(df))*100,2),' %')

the percentage of total :  0.35  %


In [115]:
## Choose mediaType

mediaType = 'Electricity'

df_temp = df[df['mediaType']==mediaType]
temp = anomalies[anomalies['mediaType']==mediaType]
print('locations with anomalies and their percentage : \n')
print(temp['Location_'].value_counts().index,'\n')
print(np.round(temp['Location_'].value_counts().to_numpy()*100,2)/len(temp))

locations with anomalies and their percentage : 
Index(['India kalwa', 'China', 'Netherland', 'Austria', 'India GU', 'Portugal',
       'Switzerland Zug', 'United State FH', 'United State JS',
       'United State SQ', 'Switzerland SH', 'United State DA'],
      dtype='object')
[71.77700348 13.41463415  6.62020906  2.26480836  1.91637631  1.04529617
  0.87108014  0.69686411  0.69686411  0.34843206  0.17421603  0.17421603]


In [116]:
print('the percentage of anomalies in this mediaType : ',round((len(temp)/len(df_temp))*100,2),' %')

0.31

In [117]:
print('the number of meterIds : ',temp.meterId.nunique())

the number of meterIds :  62


In [48]:
print("****The anomalies in this mediaType****\n")
temp

,date,meterId,Country,mediaType,data,unit,Location_,meterType,year,month,day,dayofyear,weekofyear,weekday
595,01.08.2021,2409288,India,ElectricityGenEmergency,0.0,kWh,India kalwa,Main meter,2021,1,8,8,1,4
596,01.11.2021,2409288,India,ElectricityGenEmergency,0.0,kWh,India kalwa,Main meter,2021,1,11,11,2,0
597,23.11.2021,2409288,India,ElectricityGenEmergency,0.0,kWh,India kalwa,Main meter,2021,11,23,327,47,1
598,03.01.2022,2409288,India,ElectricityGenEmergency,0.0,kWh,India kalwa,Main meter,2022,3,1,60,9,1


## Anomaly detection using DBSCAN
We are going to use the same parameters as the other study namely : eps = 1, min_samples = 2, to conserve the coherence of the results

In [33]:
## Performing DBSCAN

X = df1.to_numpy()

model = DBSCAN(eps = 1, min_samples = 10, metric = 'euclidean').fit(X)
pred = model.fit_predict(X)
idxs = []

for i, e in enumerate(model.labels_):
    if e == -1 :
        idxs.append(i)

anomalies = df.filter(items = idxs, axis=0)
anomalies = anomalies.reset_index(drop=True)
anomalies

,date,meterId,Country,mediaType,data,unit,Location_,meterType,year,month,day,dayofyear,weekofyear,weekday
0,01.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,1,10,10,2,3
1,02.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,2,10,41,6,6
2,03.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,3,10,69,10,6
3,04.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,4,10,100,15,2
4,05.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,5,10,130,19,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283302,12.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,12,6,340,49,0
283303,13.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,6,13,164,23,6
283304,14.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,6,14,165,24,0
283305,15.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,6,15,166,24,1


In [34]:
print('the percentage of total : ',round((len(anomalies)/len(df))*100,2),' %')

the percentage of total :  97.72  %


In [35]:
## Choose mediaType

mediaType = 'Electricity'

df_temp = df[df['mediaType']==mediaType]
temp = anomalies[anomalies['mediaType']==mediaType]
print('locations with anomalies and their percentage : \n')
print(temp['Location_'].value_counts().index,'\n')
print(np.round(temp['Location_'].value_counts().to_numpy()*100,2)/len(temp))

locations with anomalies and their percentage : 

Index(['India kalwa', 'Austria', 'Switzerland Zug', 'Portugal', 'Germany',
       'Netherland', 'United State HU', 'United State DA', 'India GU',
       'United State HE', 'United State SQ', 'United State FH', 'China',
       'Denmark', 'Switzerland SH', 'United State JS', 'United State PO',
       'United State LA', 'United State BE', 'Singapour', 'India BAN',
       'India JI'],
      dtype='object') 

[32.69834278 24.12707152 10.71133623  8.84765733  5.20508727  4.29004019
  4.24984859  1.53388757  1.23878214  0.92715961  0.92495733  0.91339536
  0.79942741  0.53625502  0.53625502  0.53240104  0.48340032  0.46523151
  0.46302924  0.18389033  0.16627209  0.16627209]


In [36]:
print('the percentage of anomalies in this mediaType : ',round((len(temp)/len(df_temp))*100,2),' %')

the percentage of anomalies in this mediaType :  98.26  %


In [37]:
print('the number of meterIds : ',temp.meterId.nunique())

the number of meterIds :  244


In [38]:
print("****The anomalies in this mediaType****\n")
temp

****The anomalies in this mediaType****



,date,meterId,Country,mediaType,data,unit,Location_,meterType,year,month,day,dayofyear,weekofyear,weekday
0,01.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,1,10,10,2,3
1,02.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,2,10,41,6,6
2,03.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,3,10,69,10,6
3,04.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,4,10,100,15,2
4,05.10.2019,1131988,Austria,Electricity,3682.14,kWh,Austria,Submeter,2019,5,10,130,19,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283302,12.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,12,6,340,49,0
283303,13.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,6,13,164,23,6
283304,14.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,6,14,165,24,0
283305,15.06.2021,6348613,United States,Electricity,4.34,kWh,United State DA,Main meter,2021,6,15,166,24,1
